# Consumption Report

After obtaining all the metrics from the best model, it's time to present a summary of the results obtained with the best algorithm trained with a specific time and different interpolation method.

Note that all prediction results are saved in the path: C:/Users/USUARIO/jupyter_notebooks/segundo_experimento_datos_acoplados_temporalmente-main/data/NILM/Results

Additionally, besides presenting the results in this notebook, we save them in:
c:/Users/USUAIO/jupyter_notebooks/segundo_experimento_datos_acoplados_temporalmente-main/data/results.htm

First, we need to load libraries like plotly, nilmtk, pandas, datetime and codecs


In [2]:
import pandas as pd
import datetime
import codecs
from plotly.io import to_html
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px 
from plotly.subplots import make_subplots
from nilmtk import DataSet
import warnings
from IPython.display import SVG

warnings. filterwarnings("ignore")
plt.style.use('bmh')


## Brief explanation of the reporting function
We create a function that retrieves predictions from the best model and calculates the total energy and power consumed from it.

Additionally, it provides other interesting data such as the start and end period, sampling rate, etc.

We also generate a graph showing disaggregated consumption of different appliances compared to the main meter reading.

Notably, we export all relevant information to an HTML file, which will be overwritten each time we run this notebook.

In [14]:
def report():
        
    #Tomamos la mejor prediccion, que para este segundo experimento de datos sincronizados temporalmente sin tomar los 
    #transitorios es la siguinte combinación:
    # -algoritmo FHMM  
    # -el mejor tiempo de muestreo(60seg) 
    # -el mejor metodo de interpolacion (metodo first)
    
 
    results = DataSet('./data/NILM/Results/CO_1s_mean_pred.h5')   
    #ruta del fichero de salida 
    html_dir =('results.htm')   

    
    labels = [ 'Electric furnace', 'Microwave','Television','Kettle','Vacuum cleaner','Electric space heater','Electric shower heater','Fan','Fridge','Freezer'] #etiquetas
    
    results_elec = results.buildings[1].elec
    
    print (results_elec) #pintamos los diferentes aplicativos con sus asociaciones
   
   
    #pie
    df1=pd.DataFrame(results_elec.submeters().fraction_per_meter(), columns=['Fracción'])
    df1['Energía']=round(df1['Fracción']*results_elec.total_energy()[0],1)

    
    fig1 = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
    fig1.add_trace(go.Pie(values=df1['Fracción'], name='Porcentaje', hoverinfo="label+percent+name",
                          textinfo='label+percent', labels=labels, sort=False), 1, 1)
    fig1.add_trace(go.Pie(values=df1['Energía'], name='Energía', hoverinfo="label+value+name", textinfo='label+value',
                          labels=labels, sort=False, texttemplate="%{value:.1f} kWh"), 1, 2)
    fig1.update_traces(hole=.3)
    fig1.update_layout(width=900, height=450,
         annotations=[dict(text='%', x=0.21, y=0.5, font_size=22, showarrow=False),
                      dict(text='kWh', x=0.81, y=0.5, font_size=22, showarrow=False)])
    
    fig1.show() #pintamos el resultado por pantalla
    pie=fig1.to_html()# convertimos el grafico en un objeto para poderlo copiar en un fichero html
   
    
    #lines
    df2=results_elec.select(instance=[2,3,4,5,6,7,8,9,10,11]).dataframe_of_meters(ac_type='active')
    df2.columns=labels[:10] #old 5
    fig2=px.line(df2, width=900, height=500,labels={'variable':'Dispositivos', 'index':'Fecha', 'value':'Potencia (W)'})
  
    fig2.show() #pintamos el resultado por pantalla
    lines=fig2.to_html() # convertimos el grafico en un objeto para poderlo copiar en un fichero html
   
    
    #metadata
    date=results.metadata['date'] #recuperamos del dataset la fecha
    date=datetime.datetime.strptime(date,'%Y-%m-%dT%H:%M:%S').strftime('%d/%m/%Y %H:%M:%S')
    
    sample_period=results.metadata['meter_devices']['mains']['sample_period'] #recuperamos del dataset el tiempo de muestreo
    sample_period =str (sample_period)+ 'segundos'
    print ('Sample_period:',sample_period)
    
    start=results.metadata['timeframe']['start'] #recuperamos del dataset la hora de inicio
    start = datetime.datetime.strptime(start, '%Y-%m-%dT%H:%M:%S%z').strftime('%d/%m/%Y %H:%M:%S')
  
  
    #start=datetime.datetime.strptime(start,'%Y-%m-%dT%H:%M:%S+01:00').strftime('%d/%m/%Y %H:%M:%S')
    print ('Start:',start)
    
    end=results.metadata['timeframe']['end'] #recuperamos del dataset la hora de finalizacion
    end=datetime.datetime.strptime(end,'%Y-%m-%dT%H:%M:%S%z').strftime('%d/%m/%Y %H:%M:%S')
    #end=datetime.datetime.strptime(end,'%Y-%m-%dT%H:%M:%S+01:00').strftime('%d/%m/%Y %H:%M:%S')

    print ('End:',end)


    
    
    #a continuacion creamos una cadena en formato HTML estatico que incluye 
    #los graficos y los datos mas relevantes recuperados del dataset    
    
    msg ="""
    <html>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css">
    <head>  <meta http-equiv="content-type" content="text/html; charset=UTF-8"> </head>
       <body>
        <h1><td><img src="logotipo.png" width="160" ></td><b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Desagregación Consumo energético con OZM y NILMTK</b></h1>
          <br>  
        <h4><b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Localización y fecha</b></h4>
      
        <table >
          <tbody>
            <tr>
            <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
             <td>Local:</td> <td>Laboratorio Electrotecnia Escuela Politecnica </td> </tr>
             <tr>
             <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
                        <td >Fecha:          </td>     <td >"""+date+""" </td>
            </tr>
          </tbody>
        </table>
        <h4><b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Energía Consumida</b></h4>
        <table >
          <tbody>
            <tr>
            <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
              <td >Fecha
                Inicio:</td>
              <td >"""+start+"""<br>
                </td>
            </tr>
            <tr>
            <td></td>
              <td >Fecha Fin:</td>
              <td >"""+end+"""<br>
                </td>
            </tr>
            <tr>
            <td></td>
              <td >Periodo de
                muestreo:</td>
              <td >"""+sample_period+"""<br>
              </td>
            </tr>
          </tbody>
        </table>
        <p></p>
        <h4><b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Gráfico de energía desagregada</b></h4>
        """+pie+"""
        <h4><b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Gráfico de potencia instantánea desagregada</b> </h4>
        """+lines+"""
        <p><br>
        </p>
        <p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Se puede encontrar mas información en  <a href="https://github.com/crn565/segundo_experimento_datos_acoplados_temporalmente">github.com/crn565
                  </a></p>
       &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Octubre 2022 by CRN<span><br>
        </span><br>
        <br>
      </body>
     </html>"""

    with codecs.open(html_dir, 'w', encoding='utf8') as f:
        f.write(msg)     #escribimos la cadena msg en el fichero de salida
        f.close()        #cerramos el fichero
    return print('!El informe  ya se ha creado!')
    
    

In [15]:
#llamada a la función
report()

MeterGroup(meters=
  ElecMeter(instance=1, building=1, dataset='CO', site_meter, appliances=[])
  ElecMeter(instance=2, building=1, dataset='CO', appliances=[Appliance(type='electric furnace', instance=1)])
  ElecMeter(instance=3, building=1, dataset='CO', appliances=[Appliance(type='microwave', instance=1)])
  ElecMeter(instance=4, building=1, dataset='CO', appliances=[Appliance(type='television', instance=1)])
  ElecMeter(instance=5, building=1, dataset='CO', appliances=[Appliance(type='incandescent lamp', instance=1)])
  ElecMeter(instance=6, building=1, dataset='CO', appliances=[Appliance(type='vacuum cleaner', instance=1)])
  ElecMeter(instance=7, building=1, dataset='CO', appliances=[Appliance(type='electric space heater', instance=1)])
  ElecMeter(instance=8, building=1, dataset='CO', appliances=[Appliance(type='electric shower heater', instance=1)])
  ElecMeter(instance=9, building=1, dataset='CO', appliances=[Appliance(type='fan', instance=1)])
  ElecMeter(instance=10, buildin

Sample_period: 1segundos
Start: 09/06/2023 12:55:00
End: 09/06/2023 13:36:00
!El informe  ya se ha creado!
